In [3]:
import openai, json, pandas as pd, os


In [4]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [5]:
def query(prompt, **kwargs):
  """
  wrapper for the API to save the prompt and the result
  """
  props = {
  "engine":"davinci",
  "temperature":0,
  "max_tokens":250,
  "stop":"\n\n",
  **kwargs
  }

  r = openai.Completion.create(prompt=prompt, **props)["choices"][0]["text"].strip()
  return r
     


Making sure this works

In [7]:
query("q: what is 1+1?\na:", stop="\n")


'2'

Let's go and start creating some frew shots from the 

In [10]:
train = pd.read_json("anli_v1.0/R3/train.jsonl",lines=True)


In [13]:
train.label.value_counts()

n    40778
e    32292
c    27389
Name: label, dtype: int64

In [14]:
# will grab 2 random neutral, entailment, and contradiction examples

entailment = train[train.label=="e"].sample(2)
neutral = train[train.label=="n"].sample(2)
contradiction = train[train.label=="c"].sample(2)

In [24]:
lookupDict = {
    'e': 'entailment',
    'n': 'neutral',
    'c': 'contradiction'
}

def createExample(row):
  return f"""{{
    "Background": "{row.context}",
    "Claim": "{row.hypothesis}",
    "Relationship": "{lookupDict[row.label]}"
}}"""

In [25]:
[print(createExample(row)) for _, row in entailment.iterrows()]
[print(createExample(row)) for _, row in neutral.iterrows()]
[print(createExample(row)) for _, row in contradiction.iterrows()]

{
    "Background": "Luciano Chailly (born Ferrara, January 19, 1920 – died Milan, December 24, 2002) was an Italian composer and arts administrator of French descent. He was the father of harpist Cecilia Chailly, conductor Riccardo Chailly and journalist Floriana Chailly. As a composer, Chailly was best known for his operas, many of which were composed to libretti by Dino Buzzati.",
    "Claim": "Luciano Chailly was born in January",
    "Relationship": "entailment"
}
{
    "Background": "At least 58 people are now dead as a result of the recent flooding in Yemen, and at least 20,000 in the country have no access to shelter. Five people are also reported missing. The Yemeni government has pledged to send tents to help the homeless. The flooding is caused by the recent heavy rain in Yemen, which came as a shock due to the fact that the country only receives several centimeters of rain per year.",
    "Claim": "Rising water levels in Yemen has resulted in people passing away",
    "Rela

[None, None]

Let's start by figuring out the inconsistncy bit

In [44]:
example = """You are an expert analyst evaluating where claims are contradictions, entailments, or neutral with respect to a background. You are given a claim and a background. You must decide whether the claim is a contradiction, entailment, or neutral with respect to the background. You can use the claim and background.
{
"Background": "The others gave in very soon , and longed to be friends , for now there was no Daisy to pet and cook for them ; no Nan to amuse and doctor them ; and , worst of all , no Mrs. Jo to make home life pleasant and life easy for them .<br>To their great affliction , Mrs. Jo seemed to consider herself one of the offended girls , for she hardly spoke to the outcasts , looked as if she did not see them when she passed , and was always too busy now to attend to their requests .",
"Claim": "Nan was there to doctor them.",
"Part of Claim in Background": "no Nan to amuse and doctor them",
"Evaluation": "In the claim, Nan is there to doctor them. In the background, it is stated that there is no Nan to doctor them. This is inconsistent."
"Verdict": "Contradiction"
},
{
"Background": "How to avoid eviction<br>Make timely payments. The easiest way to avoid eviction is to pay your rent on time. Read your lease to see if there is any grace period (such as three or five days).",
"Claim": "most leases have grace periods for payment",
"Part of Claim in Background": "if there is any grace period (such as three or five days)",
"Evaluation": "In the claim, most leases have grace periods for payment. In the background, it is stated that there may be a grace period of three or five days. They neither contradict nor entail each other so it's neutral."
"Vedict": "Neutral"
},
{
"Background": "At least 58 people are now dead as a result of the recent flooding in Yemen, and at least 20,000 in the country have no access to shelter. Five people are also reported missing. The Yemeni government has pledged to send tents to help the homeless. The flooding is caused by the recent heavy rain in Yemen, which came as a shock due to the fact that the country only receives several centimeters of rain per year.",
"Claim": "Rising water levels in Yemen has resulted in people passing away",
"Part of Claim in Background": "At least 58 people are now dead as a result of the recent flooding in Yemen",
"Evaluation": "In the claim, rising water levels in Yemen has resulted in people passing away. In the background, it is stated that at least 58 people are now dead as a result of the recent flooding in Yemen. This entails the claim.",
"Verdict": "Entailment"
},
{
"Background": "I wonder how he expects us to put forward a strong position on these issues if we are not part of the World Trade Organization or if we were not in a position, with meetings like Seattle, where we can pull together different countries from around the world that have a similar position to ours.",
"Claim": "there were never meetings in Seattle",
"Part of Claim in Background": "with meetings like Seattle",
"Evaluation": "In the claim, there were never meetings in Seattle. In the background, it is stated that there were meetings like Seattle. This is inconsistent.",
"Verdict": "Contradiction"
},
{
"Background": "Luciano Chailly (born Ferrara, January 19, 1920 – died Milan, December 24, 2002) was an Italian composer and arts administrator of French descent. He was the father of harpist Cecilia Chailly, conductor Riccardo Chailly and journalist Floriana Chailly. As a composer, Chailly was best known for his operas, many of which were composed to libretti by Dino Buzzati.",
"Claim": "Luciano Chailly was born in January",
"Part of Claim in Background": "January 19, 1920",
"Evaluation": "In the claim, Luciano Chailly was born in January. In the background, it is stated that he was born on January 19, 1920. This entails the claim.",
"Verdict": "Entailment"
},
{
"Background": "Oil Vials<br>I purchased several vials of perfumed oils. I picked all sweet scents. They were very realistic, and I was quite pleased. I put some on first thing in the morning. My husband complimented me, as did everyone in the office that day.",
"Claim": "The vials are large.",
"Part of Claim in Background": "None",
"Evaluation": "In the claim, the vials are large. In the background, there is no mention of the size of the vials. This is neutral.",
"Verdict": "Neutral"
}
"""

In [36]:
query(example, engine="text-davinci-003")

'"Part of Claim in Background": None\n"Evaluation": "In the claim, the vials are large. In the background, there is no mention of the size of the vials. This is neutral."\n"Verdict": "Neutral"\n}}'